In [ ]:
#!pip install transformers
#!pip install datasets

In [ ]:
import os
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split




import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader
import albumentations as album

from PIL import Image


In [ ]:

#upload the token from kaggle
from google.colab import files
files.upload()


In [ ]:
# Create a directory for Kaggle and move the API key there
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Set appropriate permissions on the API key file
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d emmarex/plantdisease -p /content/dataset

In [ ]:
#unzip the dataset
with zipfile.ZipFile('/content/dataset/plantdisease.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

In [ ]:
#data set path
data_path='/content/dataset/PlantVillage'
os.chdir(data_path)

In [ ]:
classes=list(os.listdir())
classes

In [ ]:
#len(list(os.listdir(os.path.join(data_path,classes[0]))))
bar_dict={}
for c in classes:
  bar_dict[c]=len(list(os.listdir(os.path.join(data_path,c))))
bar_dict



In [ ]:
sns.barplot(x=list(bar_dict.keys()),y=list(bar_dict.values()))
plt.xticks(rotation=90)
plt.show()


In [ ]:
#lets create a dataframe that have all the information that we have on the images the image path and the class

dataframe_list=[]
for index,c in enumerate(classes):
  for imag_name in list(os.listdir(os.path.join(data_path,c))):
    image_path=os.path.join(os.path.join(data_path,c,imag_name))
    dataframe_list.append([image_path,index,c])

datainf=pd.DataFrame(dataframe_list,columns=['image_path','label_id','class'])


In [ ]:
datainf

In [ ]:
train_df, test_df = train_test_split(datainf, test_size=0.2, random_state=42)
train_df,val_df = train_test_split(train_df, test_size=0.125, random_state=42)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate the sizes of train, test, and validation sets
total_samples = len(datainf)
train_size = len(train_df)
test_size = len(test_df)
val_size = len(val_df)

# Calculate the percentages
train_percent = (train_size / total_samples) * 100
test_percent = (test_size / total_samples) * 100
val_percent = (val_size / total_samples) * 100

# Data for the pie chart
sizes = [train_percent, test_percent, val_percent]
labels = ['Train', 'Test', 'Validation']

# Create the pie chart using Seaborn
plt.figure(figsize=(8, 6))
sns.set_palette("pastel")
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Dataset Split Percentage')

# Show the plot
plt.show()


In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class ImageClassificationDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data.iloc[idx, 0]
        label = self.data.iloc[idx, 1]

        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
from torchvision import transforms

transform_function=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = ImageClassificationDataset(train_df, transform=transform_function)
val_dataset = ImageClassificationDataset(val_df, transform=transform_function)
test_dataset = ImageClassificationDataset(test_df, transform=transform_function)

# Create DataLoader instances for training, validation, and testing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)


In [ ]:
from transformers import SegformerFeatureExtractor, SegformerForImageClassification,SegformerConfig
from PIL import Image
import requests
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

configuration = SegformerConfig()
#feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/mit-b5")
model = SegformerForImageClassification(configuration).from_pretrained("nvidia/mit-b5")
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)
model.to(device)


In [ ]:
from datasets import load_metric

# Load the accuracy metric
metric = load_metric("accuracy")

# Define the number of epochs
num_epochs = 15

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)

        # Calculate loss
        loss = loss_function(outputs.logits, labels)

        # Backpropagation
        loss.backward()
        optimizer.step()

        # Update metrics
        total_loss += loss.item()
        _, predicted = torch.max(outputs.logits, 1)
        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    # Calculate metrics for the epoch
    epoch_loss = total_loss / len(train_loader)
    epoch_accuracy = total_correct / total_samples

    # Print epoch results
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

    # Validation loop
    model.eval()
    val_correct = 0
    val_samples = 0

    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(images)

        _, predicted = torch.max(outputs.logits, 1)
        val_correct += (predicted == labels).sum().item()
        val_samples += labels.size(0)

    val_accuracy = val_correct / val_samples
    print(f"Validation Accuracy: {val_accuracy:.4f}")

# Test loop
model.eval()
test_correct = 0
test_samples = 0



In [ ]:

for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(images)

    _, predicted = torch.max(outputs.logits, 1)
    test_correct += (predicted == labels).sum().item()
    test_samples += labels.size(0)

test_accuracy = test_correct / test_samples
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
outputs

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")


In [ ]:
# transformss = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.RandomHorizontalFlip(p=0.5),
#     transforms.ColorJitter(brightness=0.2,
#                            contrast=0.2,
#                            saturation=0.2,
#                            hue=0.1),
#     transforms.RandomRotation(degrees=15),
#     transforms.RandomAffine(degrees=0,
#                             translate=(0.1, 0.1),
#                             scale=(0.8, 1.2)),
#     transforms.RandomResizedCrop((224, 224),
#                                  scale=(0.8, 1.0)),
#     transforms.GaussianBlur(kernel_size=3),
#     transforms.ToTensor()
# ])
